In [5]:
url1="https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
url2="https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"

In [3]:
import requests
from os.path import basename

In [37]:
for url in [url1, url2]:
    r=requests.get(url)

    with open (basename(url), "wb") as fid:
        fid.write(r.content)

In [38]:
!file *.csv

time_series_covid19_confirmed_US.csv: ASCII text, with very long lines
time_series_covid19_deaths_US.csv:    ASCII text, with very long lines


In [1]:
import pandas as pd

In [6]:
dfd=pd.read_csv(basename(url1))
dfc=pd.read_csv(basename(url2))

In [7]:
for df in [dfd,dfc]:
    cols=df.columns.tolist()
    print(len(cols), cols[0:20])

383 ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20']
382 ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_', 'Combined_Key', '1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20', '1/30/20']


Both cases and deaths have the same location metadata (except that Deaths don't come with population)
and can therefore easily be stored in the same DB.

In [8]:
dfd.iloc[0,0:13]

UID                           84001001
iso2                                US
iso3                               USA
code3                              840
FIPS                              1001
Admin2                         Autauga
Province_State                 Alabama
Country_Region                      US
Lat                            32.5395
Long_                         -86.6441
Combined_Key      Autauga, Alabama, US
Population                       55869
1/22/20                              0
Name: 0, dtype: object

In [9]:
dfc.iloc[0,0:13]

UID                           84001001
iso2                                US
iso3                               USA
code3                              840
FIPS                              1001
Admin2                         Autauga
Province_State                 Alabama
Country_Region                      US
Lat                            32.5395
Long_                         -86.6441
Combined_Key      Autauga, Alabama, US
1/22/20                              0
1/23/20                              0
Name: 0, dtype: object

In [11]:
df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,5184,5198,5227,5257,5270,5327,5358,5376,5407,5440
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,16251,16346,16513,16653,16798,16981,17128,17256,17333,17496
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1729,1730,1738,1760,1778,1793,1805,1827,1834,1882
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,2151,2162,2170,2188,2198,2212,2223,2223,2229,2247
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,5304,5308,5320,5376,5411,5439,5462,5473,5485,5517


In [68]:
#df[df["UID"] == 84001001]
#print(df.loc[0,"Combined_Key"])
print(dfc.columns[0:20])

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20'],
      dtype='object')


Dataframe contains one column per day, and one row per location.  This configuration is hard to work with.

Proposed data pipeline:
- Store COVID-19 data in relational database
- Set up scripts that retrieves the new data daily from the github and repopulates the database.

Database structure:
- Location table with location info (prinary key UID, iso1+2, code3, country, ...)
- This table can be shared by confirmed cases and deaths
- Case and death tables with covid-19 confirmed cases and deaths, respectively, or one table with additional column of count type (confirmed/death)
- Each day and location is a new entry in the cases and death tables
- Query DB for cases as function of time, per location or aggregated over regions, states, country ...

        create table locations (UID integer primary key, 
            iso2 text, iso3 text, FIPS float, Admin2 Text, 
            Province_State TEXT, Country_Region TEXT,
            );
            
        create table records (rID integer primary key references locations(UID),
            rdate date not null, ctype TEXT not null);

In [51]:
!dropdb covid19

In [52]:
!createdb covid19 "COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University"

In [12]:
len(df)

3340

In [31]:
df[df["Province_State"] == "California"]
#df[df["Admin2"] == "San Diego"]
#df[df["FIPS"] == 6067.0]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21
195,84006001,US,USA,840,6001.0,Alameda,California,US,37.646294,-121.892927,...,68016,67375,67952,68649,69107,69554,69693,70334,70823,71298
196,84006003,US,USA,840,6003.0,Alpine,California,US,38.596786,-119.822359,...,73,73,73,73,73,73,73,73,73,74
197,84006005,US,USA,840,6005.0,Amador,California,US,38.445831,-120.656960,...,2986,3001,3016,3024,3036,3038,3042,3049,3067,3074
198,84006007,US,USA,840,6007.0,Butte,California,US,39.667278,-121.600525,...,9359,9411,9474,9528,9582,9643,9711,9757,9820,9865
199,84006009,US,USA,840,6009.0,Calaveras,California,US,38.205371,-120.552913,...,1596,1608,1617,1621,1637,1647,1672,1695,1704,1713
200,84006011,US,USA,840,6011.0,Colusa,California,US,39.178820,-122.233173,...,1691,1691,1733,1796,1853,1854,1854,1854,1871,1907
201,84006013,US,USA,840,6013.0,Contra Costa,California,US,37.919235,-121.928953,...,52146,52547,52965,53349,53844,54140,54416,54951,55245,55593
202,84006015,US,USA,840,6015.0,Del Norte,California,US,41.742283,-123.897406,...,857,861,866,867,869,870,873,873,874,878
203,84006017,US,USA,840,6017.0,El Dorado,California,US,38.779660,-120.523317,...,7575,7659,7826,7886,7949,8002,8002,8002,8184,8245
204,84006019,US,USA,840,6019.0,Fresno,California,US,36.757339,-119.646695,...,81294,82039,82485,83200,83591,84137,84744,85274,85808,86336


In [36]:
#df[df["Admin2"] == "Out of CA"].iloc[0,11:].max()
df[df["Admin2"] == "Out of CA"].iloc[0,0:20]

UID                                84080006
iso2                                     US
iso3                                    USA
code3                                   840
FIPS                                  80006
Admin2                            Out of CA
Province_State                   California
Country_Region                           US
Lat                                       0
Long_                                     0
Combined_Key      Out of CA, California, US
1/22/20                                   0
1/23/20                                   0
1/24/20                                   0
1/25/20                                   0
1/26/20                                   0
1/27/20                                   0
1/28/20                                   0
1/29/20                                   0
1/30/20                                   0
Name: 225, dtype: object

In [49]:
from ipyleaflet import Map, Marker, MarkerCluster

m = Map(center=(37, -120), zoom=5)

markers=[]

tdf=df[df["Province_State"] == "California"]

for n in tdf.index:
    lon=tdf.loc[n,"Long_"]
    lat=tdf.loc[n,"Lat"]
    markers.append(Marker(location=(lat,lon)))

marker_cluster = MarkerCluster(markers=markers)

m.add_layer(marker_cluster);

m

Map(center=[37, -120], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [47]:
marker_cluster

MarkerCluster(markers=(Marker(location=[-121.8929271, 37.64629437], options=['alt', 'draggable', 'keyboard', '…

In [42]:
tdf.index

Int64Index([], dtype='int64')